Adult data set from uci website read and column names initialized (Contains Categorical and Continuous data)

In [ ]:
import pandas as pd
import numpy as np

df=pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",header=None)

names=['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','salary']
df.columns=names

list0 gives columns having values '?'

Each column of DataFrame is a series and each '?' in a series is replaced by the mode of that series (most occuring value in that series)

The above tasks fill the missing data in Adult dataset 

In [ ]:
list0=[]
for column in df.columns:
    try:
        data= df[df[column]==' ?']
        if data.shape[0]>0:
            list0.append(column)
            
    except:
        continue;

for i in list0:
    df[i].replace(' ?',df[i].mode()[0], inplace=True)

Train-Test Split the message data using sklearn

X_train gives Training message data on which algorithm is trained on

X_test gives Testing message data on which algorithm is tested on

y_train gives output column for prediction derived from training data

y_train gives output column to be checked for prediction derived from testing data and calculate accuracy of algorithm

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[df.columns.drop('salary')], df['salary'], test_size=0.30)

XTrain=X_train.reset_index(drop=True)
YTrain=y_train.reset_index(drop=True)
ytest=y_test.reset_index(drop=True)
xtest=X_test.reset_index(drop=True)

XTrain1 gives continuous training data of XTrain
xtest1 gives continuous testing data of xtest

XTrain2 gives categorical training data of XTrain
xtest2 gives categorical testing data of xtest

In [ ]:
XTrain1=XTrain[XTrain._get_numeric_data().columns]
YTrain1=YTrain
xtest1=xtest[xtest._get_numeric_data().columns]
ytest1=ytest

XTrain2=XTrain[list(set(XTrain.columns)-set(XTrain._get_numeric_data().columns))]
YTrain2=YTrain
xtest2=xtest[list(set(XTrain.columns)-set(xtest._get_numeric_data().columns))]
ytest2=ytest

dfTRAIN1 has training data

data['Mean'] is mean of given column values corresponding to an output value (eg: age <=50K) i.e in same row
data['Var'] is variance of given column values corresponding to an output value (eg: age <=50K) i.e in same row

all this is stored in Dict2 key is 'column_name''Output_Value' and value is mean and variance of those values of column_name corresponding to Output_Value (in same row)

This the role of Fit1 function

In [ ]:
def Fit1(XTrain1,YTrain1):
    Dict2={}
    
    dfTRAIN1=XTrain1.copy()
    dfTRAIN1['salary']=YTrain1 
    
    for i in YTrain1.value_counts().index:
        for j in XTrain1.columns:
            data={}
            mean=np.mean(dfTRAIN1[j][dfTRAIN1['salary']==i].values)
            variance=np.var(dfTRAIN1[j][dfTRAIN1['salary']==i].values)
            data['Mean']=mean
            data['Var']=variance
            Dict2[j+i]=data

    return(Dict2)       

dfFit has categorical training data. Probabilities of each 'word' given an 'Output_Value' is stored in Dict2 
Fit2 function returns this Dictionary of probabilities

In [ ]:
def Fit2(XTrain2, YTrain2):
    Dict1 = {}
    Dict2 = {}

    dfFit=XTrain2.copy()
    dfFit['salary']=YTrain2

    for k in XTrain2.columns.values:
        for i in XTrain2[k].value_counts().index:
            for j in YTrain2.value_counts().index:
                Prob_Denom=dfFit[dfFit['salary']==j].shape[0]/dfFit.shape[0]
        
                Prob_Numer=dfFit[(dfFit[k]==i)&(dfFit['salary']==j)].shape[0]/dfFit.shape[0]

                Dict2[i+j]=Prob_Numer/Prob_Denom
            
        Dict1[k]=Dict2
    
    return(Dict1)

In [ ]:
Dict1={}

Dict1=Fit1(XTrain1,YTrain1)

In [ ]:
Dict4={}
Dict4=Fit2(XTrain2, YTrain2)

words in testing data but not in training data are omitted

probabilities for Prob1(word|Output_Value) for numerical data is calculated using GaussianNB formula  

probabilities for Prob2(word|Output_Value) for categorical data is retreived from Dictionary Dict4

These are all then multiplied for attributes of each row Naive_Bayes style to give prediction for a row

In [ ]:
import math

def predict(xtest1,ytest1,Dict1,xtest2,ytest2,Dict4):
    predict=[]
    for i in range(0,xtest1.shape[0]):
        print(i)
        great=0.0
        greatest='a'
        
        for j in ytest1.value_counts().index:
            val1=1.0
            val2=1.0
            val=1.0
            for k1 in xtest1.columns:
                try:
                    x=Dict1[k1+j]
                    mean=x['Mean']
                    variance=x['Var']
                    Prob1=(1.0/math.sqrt(2.0*math.pi*(variance**2)))*math.exp(-1.0*(((xtest1[k1][i]-mean)**2)/(2.0*(variance**2))))
                
                except KeyError:
                    print('Hey')
                    continue
                val1=val1*Prob1
            
            for k2 in xtest2.columns:
                try:
                    Prob2=Dict4[k2][xtest2[k2][i]+j]
                except KeyError:
                    print('Hey1')
                    continue
                val2=val2*Prob2
            
            val=val1*val2*((ytest1[ytest1==j].shape[0])/ytest1.shape[0])
                 
            if(val>great):
                great=val
                greatest=j
             
        predict.append(greatest)
                
    return predict    
                    

In [ ]:
predict_list=predict(xtest1,ytest1,Dict1,xtest2,ytest2,Dict4)

Accuracy calculated based on predictions returned and actual output values of test data

Accuracy is 75.52%

In [ ]:
def accuracy(predict_list,ytest1):
    count=0
    for i in range(0,ytest1.shape[0]):
        if(predict_list[i]==ytest1[i]):
            count=count+1
    
    Accuracy=count/ytest1.shape[0]
    return Accuracy

In [12]:
print((accuracy(predict_list,ytest1))*100,"%")

75.52461869178012 %
